In [7]:
from pyspark.sql.functions import explode, sequence, to_date
from pyspark.sql.functions import col, year, month, dayofmonth, weekofyear, date_format, to_date, expr

StatementMeta(, ec990d2c-0ce7-4055-9c11-5eaabd5af800, 65, Finished, Available, Finished)

In [8]:
# Parameter
beginDate = '2023-01-01'
endDate = '2030-12-31'
display_data = False

StatementMeta(, ec990d2c-0ce7-4055-9c11-5eaabd5af800, 66, Finished, Available, Finished)

In [9]:
df = spark.sql(f"select explode(sequence(to_date('{beginDate}'), to_date('{endDate}'), interval 1 day)) as date")

StatementMeta(, ec990d2c-0ce7-4055-9c11-5eaabd5af800, 67, Finished, Available, Finished)

In [10]:
date_df = df.select(
    date_format("date","yyyyMMdd").alias("DateKey").cast("int"),
    date_format("date","yyyy-MM-dd").alias("Date"),
    col("date").alias("Date2Key"),
    year("date").alias("Year"),
    month("date").alias("Month"),
    dayofmonth("date").alias("Day"),
    weekofyear("date").alias("WeekOfYear"),
    date_format("date","yyyy-MM").alias("YearMonth"),
    date_format("date", "E").alias("DayOfWeek")
)

date_df = date_df.createOrReplaceTempView('calendar_temp')

query = """
    SELECT 
        *
        ,DAYOFWEEK(date) AS DayOfWeekNum
        ,CASE WHEN ( YEAR(date) = YEAR(CURRENT_DATE()) ) THEN 1 ELSE 0 END  AS IsCurrentYear
        ,CASE WHEN ( YEAR(date) = YEAR(CURRENT_DATE())-1 ) THEN 1 ELSE 0 END  AS IsPreviousYear
        ,CASE WHEN ( YEAR(date) = YEAR(CURRENT_DATE()) AND QUARTER(date) = QUARTER(CURRENT_DATE()) ) THEN 1 ELSE 0 END  AS IsCurrentQuarter
        ,CASE WHEN ( YEAR(date) = YEAR(CURRENT_DATE()) ) THEN 1 ELSE 0 END  AS IsCurrentMonth
        ,CASE WHEN ( DATE_FORMAT(date, 'yyyy-MM') = DATE_FORMAT(ADD_MONTHS(CURRENT_DATE(), -1), 'yyyy-MM') ) THEN 1 ELSE 0 END  AS IsPreviousMonth
        ,CASE WHEN ( date BETWEEN DATE_ADD(CURRENT_DATE(), -14) AND CURRENT_DATE() ) THEN 1 ELSE 0 END  AS IsInLast14Days
        ,CASE WHEN ( date BETWEEN DATE_ADD(CURRENT_DATE(), -30) AND CURRENT_DATE() ) THEN 1 ELSE 0 END  AS IsInLast30Days
    FROM calendar_temp
"""

final_date_df = spark.sql(query)


StatementMeta(, ec990d2c-0ce7-4055-9c11-5eaabd5af800, 68, Finished, Available, Finished)

In [11]:
if display_data:
    display(final_date_df)

StatementMeta(, ec990d2c-0ce7-4055-9c11-5eaabd5af800, 69, Finished, Available, Finished)

In [12]:
# Write the DataFrame to the lakehouse
final_date_df.write.mode("overwrite").option("mergeSchema", "true").format("delta").saveAsTable("calendar")

StatementMeta(, ec990d2c-0ce7-4055-9c11-5eaabd5af800, 70, Finished, Available, Finished)

In [13]:
%%sql
SELECT * FROM calendar

StatementMeta(, ec990d2c-0ce7-4055-9c11-5eaabd5af800, 71, Finished, Available, Finished)

<Spark SQL result set with 1000 rows and 17 fields>